Pandas -> 1 - 5Gb >>>  Chunksize -> 5 - 30Gb

Dask -> 30 - 200Gb

Spark(Scala), PySpark API -> 1000Gb ~ 1Tb, 1000Tb ~ 1Pb 


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/csv/car_evaluation.csv', header=None)
# df = pd.read_csv('../data/csv/us_companies.csv', chunksize=1000)
df.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
df.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

In [4]:
df.describe()

,buying,maint,doors,persons,lug_boot,safety,class
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,432,576,576,576,1210


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [19]:
df["doors"].replace({"5more":6}, inplace=True)
df["persons"].replace({"more":10}, inplace=True)

df.to_csv("../data/csv/car_evaluation_processed.csv")

In [7]:
import dask.array as da
a = da.random.randint(10,100,100000000).reshape(1000000,100)
a.compute()

array([[34, 88, 20, ..., 70, 60, 26],
       [73, 77, 21, ..., 39, 74, 20],
       [92, 63, 77, ..., 11, 72, 17],
       ...,
       [89, 54, 19, ..., 25, 11, 67],
       [13, 15, 84, ..., 63, 23, 88],
       [22, 56, 28, ..., 17, 82, 25]])

In [20]:
import dask.dataframe as dd

In [21]:
# data = dd.read_csv("../data/csv/car_evaluation.csv", dtype={'buying': str, "maint": str, "num_doors": int, "num_person": int,"lug_boot": str, "safety": str, "decision": str})
# data = dd.read_csv("../data/csv/car_evaluation.csv", header=None)
data = dd.read_csv("../data/csv/car_evaluation_processed.csv")
data.head()
#data.compute()

,Unnamed: 0,buying,maint,doors,persons,lug_boot,safety,class
0,0,vhigh,vhigh,2,2,small,low,unacc
1,1,vhigh,vhigh,2,2,small,med,unacc
2,2,vhigh,vhigh,2,2,small,high,unacc
3,3,vhigh,vhigh,2,2,med,low,unacc
4,4,vhigh,vhigh,2,2,med,med,unacc


In [2]:
import pyspark as spark
from pyspark import SparkContext, SQLContext

In [3]:
try:
    sc = SparkContext()
except:
    print("Spark Context is already running")
s = SQLContext(sc)

22/04/19 21:47:33 WARN Utils: Your hostname, serendipita-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 10.100.144.172 instead (on interface wlp7s0)
22/04/19 21:47:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/19 21:47:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/serendipita/Documents/utec/bigdata/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [7]:
# data = s.read.options(header=True).csv("../data/csv/car_evaluation.csv")
data = s.read.options(header=True).csv("../data/csv/car_evaluation_processed.csv")

In [8]:
data.show(5)

+---+------+-----+-----+-------+--------+------+-----+
|_c0|buying|maint|doors|persons|lug_boot|safety|class|
+---+------+-----+-----+-------+--------+------+-----+
|  0| vhigh|vhigh|    2|      2|   small|   low|unacc|
|  1| vhigh|vhigh|    2|      2|   small|   med|unacc|
|  2| vhigh|vhigh|    2|      2|   small|  high|unacc|
|  3| vhigh|vhigh|    2|      2|     med|   low|unacc|
|  4| vhigh|vhigh|    2|      2|     med|   med|unacc|
+---+------+-----+-----+-------+--------+------+-----+
only showing top 5 rows



22/04/19 21:48:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , buying, maint, doors, persons, lug_boot, safety, class
 Schema: _c0, buying, maint, doors, persons, lug_boot, safety, class
Expected: _c0 but found: 
CSV file: file:///home/serendipita/Documents/utec/bigdata/utec-bigdata-2022-1/data/csv/car_evaluation_processed.csv


In [9]:
data.select(['maint', 'buying']).show()

+-----+------+
|maint|buying|
+-----+------+
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
+-----+------+
only showing top 20 rows

